# Backup e migrazione di Redis

Questo notebook guida il salvataggio dello stato di un database Redis e il suo ripristino su un'altra istanza specificando indirizzo IP e porta. Il flusso serializza ogni chiave usando i comandi nativi del tipo corrispondente (stringhe, hash, liste, set, zset) preservando automaticamente i TTL, con un fallback su `DUMP`/`RESTORE` per i tipi non supportati nativamente.

## Come usare il notebook

1. Aggiorna le configurazioni di connessione (`SOURCE_REDIS_CONFIG` e `TARGET_REDIS_CONFIG`) con IP, porta, database e credenziali delle istanze che vuoi utilizzare.
2. Esegui la sezione **Verifica delle connessioni** per controllare l'accesso ai database.
3. Avvia la sezione **Creazione del backup** per generare un file JSON contenente l'intero stato del Redis sorgente.
4. Opzionalmente trasferisci il file su un'altra macchina.
5. Esegui la sezione **Ripristino del backup** per ricreare i dati sul Redis di destinazione.

⚠️ Le celle che interagiscono con Redis effettuano operazioni reali sul database: usa l'opzione `FLUSH_TARGET_BEFORE_RESTORE` con attenzione.

In [ ]:
from pathlib import Path
from typing import Any, Dict

from redis.exceptions import ConnectionError, ResponseError

from redis_backup import (
    build_redis_client,
    create_redis_backup,
    display_backup_summary,
    get_database_overview,
    load_backup_from_file,
    restore_redis_backup,
    save_backup_to_file,
)


In [ ]:
# Configurazione delle istanze Redis
SOURCE_REDIS_CONFIG: Dict[str, Any] = {
    "host": "127.0.0.1",  # IP o hostname dell'istanza sorgente
    "port": 6379,
    "db": 0,
    "username": None,  # Imposta un valore se il server richiede ACL
    "password": None,
}

TARGET_REDIS_CONFIG: Dict[str, Any] = {
    "host": "127.0.0.1",  # IP o hostname dell'istanza di destinazione
    "port": 6380,
    "db": 0,
    "username": None,
    "password": None,
}

BACKUP_PATH = Path("redis_backup.json")  # Percorso locale del file di backup
SCAN_COUNT = 1000  # Numero di chiavi analizzate per iterazione durante lo scan
FLUSH_TARGET_BEFORE_RESTORE = False  # Imposta True per svuotare il DB di destinazione prima del ripristino


In [ ]:
# Le funzioni di supporto sono fornite dal modulo `redis_backup`.
# Puoi aprire il file `redis_backup.py` per esaminarne l'implementazione o
# riutilizzarle in altri script Python.


In [ ]:
# Verifica delle connessioni (facoltativa ma consigliata)
for label, configuration in (("Sorgente", SOURCE_REDIS_CONFIG), ("Destinazione", TARGET_REDIS_CONFIG)):
    try:
        overview = get_database_overview(configuration)
        print(
            f"{label}: Redis {overview['redis_version']} (modalità {overview['redis_mode']}) - "
            f"{overview['key_count']} chiavi nel DB {overview['db']}"
        )
    except ConnectionError as error:
        print(f"{label}: impossibile connettersi -> {error}")

In [ ]:
# Creazione del backup dal Redis sorgente
try:
    backup_payload = create_redis_backup(SOURCE_REDIS_CONFIG, scan_count=SCAN_COUNT)
    save_backup_to_file(backup_payload, BACKUP_PATH)
    print(f"Backup completato: {backup_payload['metadata']['key_count']} chiavi salvate.")
    print(f"File generato in: {BACKUP_PATH.resolve()}")
    display_backup_summary(backup_payload)
except ConnectionError as error:
    print(f"Connessione al Redis sorgente fallita: {error}")
except ResponseError as error:
    print(f"Errore durante l'esportazione: {error}")

In [ ]:
# Ripristino del backup sul Redis di destinazione
try:
    backup_payload = load_backup_from_file(BACKUP_PATH)
    restored_keys = restore_redis_backup(
        backup_payload,
        TARGET_REDIS_CONFIG,
        flush_target=FLUSH_TARGET_BEFORE_RESTORE,
    )
    print(
        f"Ripristino completato: {restored_keys} chiavi importate nel DB {TARGET_REDIS_CONFIG.get('db', 0)}."
    )
except FileNotFoundError:
    print(f"File di backup non trovato: {BACKUP_PATH}")
except ConnectionError as error:
    print(f"Connessione al Redis di destinazione fallita: {error}")
except ResponseError as error:
    print(f"Errore durante il ripristino: {error}")